In [2]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

In [3]:
import tensorflow as tf
from typing import Any, Optional

2025-12-26 17:45:56.675641: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-26 17:45:56.778600: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-26 17:45:56.806559: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-26 17:45:56.975331: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-26 17:45:58.615583: W tensorflow/compiler/tf2

In [4]:
from mobilenetv2ssd.core.config import load_config

In [5]:
main_cfg_path = "configs/train/default.yaml"
model_cfg_path = "configs/model/mobilenetv2_ssd_voc.yaml"
data_cfg_path = "configs/data/voc_224.yaml"
eval_cfg_path = "configs/eval/default.yaml"

In [6]:
config = load_config(main_cfg_path,model_cfg_path,data_cfg_path,eval_cfg_path)

In [7]:
config['train']['optimizer']

{'name': 'sgd',
 'lr': 0.001,
 'momentum': 0.9,
 'weight_decay': 0.0005,
 'nesterov': True}

In [8]:
class OptimizerFactory:
    @staticmethod
    def build(config: dict[str,Any], learning_schedule: tf.keras.optimizers.schedules.LearningRateSchedule | None = None):

        config = OptimizerFactory.build_optimizer_config(config)
        
        name = str(config['name']).strip().lower()
        
        weight_decay = float(config['weight_decay'])
        learning_rate = float(config['lr'])

        learning_schedule = learning_schedule if learning_schedule is not None else learning_rate

        if name == "sgd":
            momentum = float(config['momentum'])
            nesterov = bool(config['nesterov'])
            return tf.keras.optimizers.SGD(learning_rate = learning_schedule , momentum = momentum, nesterov = nesterov, name = name)
        elif name in ("adam","adamw"):
            beta_1 = float(config['beta1'])
            beta_2 = float(config['beta2'])
            epsilon = float(config['epsilon'])
            if name == "adam":
                return tf.keras.optimizers.Adam(learning_rate = learning_schedule, beta_1 = beta_1, beta_2 = beta_2, epsilon = epsilon, name = name)
            else:
                return tf.keras.optimizers.AdamW(learning_rate = learning_schedule, beta_1 = beta_1, beta_2 = beta_2, epsilon = epsilon, weight_decay = weight_decay, name = name)   
        else:
            raise ValueError(f"Unknown optimizer name: {name}")
                            
    @staticmethod
    def build_optimizer_config(config: dict[str, Any]):
        train_config = config['train']
        optimizer_opts = train_config['optimizer']
        optimizer_config = {
            'name': optimizer_opts.get('name', 'sgd'),
            'lr': optimizer_opts.get('lr', 0.001),
            'weight_decay': optimizer_opts.get('weight_decay', 0.0005),
            'beta1': optimizer_opts.get('beta1', 0.9),
            'beta2': optimizer_opts.get('beta2', 0.999),
            'epsilon': optimizer_opts.get('epsilon', 1e-07),
            'momentum': optimizer_opts.get('momentum', 0.9),
            'nesterov': optimizer_opts.get('nesterov', False),
        }
        return optimizer_config

In [9]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-2,decay_steps=10000,decay_rate=0.9)

In [10]:
opt = OptimizerFactory.build(config,lr_schedule)

I0000 00:00:1766789161.053432    2550 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1766789161.320344    2550 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1766789161.320472    2550 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1766789161.323968    2550 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1766789161.324114    2550 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [11]:
opt.learning_rate

<tf.Tensor: shape=(), dtype=float32, numpy=0.01>

In [12]:
opt